# Module 4: Somatic variants
In this workshop, we will work with common tools to process and analyze cancer sequencing data. Using the command line we will analyze DNA sequencing from the whole genome. The focus of this workshop will be on calling single nucelotide variants, insertion, deletions (commonly referred to as SNV/indels) as well as calling copy-number variations (CNVs). We will also annotate SNV & CNV files, such that we will known the functional consequence of these variants.

Data Source: we will be working on the CageKid samples from Module3. Whole genome sequencing and analysis can take multiple days to run, as such we have downsampled the files so that we can proceed in a timely fashion.

## Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install software

In [ ]:
# install conda
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

!rm Miniconda3-py37_4.8.2-Linux-x86_64.sh

# install GATK 4
!conda install -y -c bioconda gatk4

# install samtools
!conda install -y -c bioconda samtools

# install bcftools
!conda install -y -c bioconda bcftools

# install tabix
!conda install -y -c bioconda tabix

# install varscan2
!conda install -y -c bioconda varscan

# install control-freec
!conda install -y -c bioconda control-freec

# install bedtools
!conda install -y -c bioconda bedtools

--2022-01-31 23:36:25--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  46.4MB/s    in 1.7s    

2022-01-31 23:36:26 (46.4 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37

# Running Mutect2
Short Variant Discovery

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

gatk Mutect2 \
  -R references/human_g1k_v37.fasta \
  -I alignments/normal/normal.sorted.dup.recal.bam \
  -I alignments/tumor/tumor.sorted.dup.recal.bam \
  -normal normal \
  -tumor tumor \
  -O pairedVariants_mutect2.vcf \
  -L 9:130215000-130636000 \
  --germline-resource accessory_files/Homo_sapiens.GRCh37.gnomad.exomes.r2.0.1.sites.no-VEP.nohist.tidy.vcf.gz

Check input bam file

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants
samtools view alignments/tumor/tumor.sorted.dup.recal.bam | head -n 1 | less -S


GA13-EAS1663_0010_FC:8:103:3239:19652#0	177	1	1584600	25	67M	2	98074146	0	AGCTGGGATTACAGGCATGCGCCACCACACCCGGCTAATTTTTTTATTTTTAGTAGAGACGGGGTTT	???>???=>>>=?@?@?:??@?@@@@@@@@AAACACA@CCCAACC@CCACACA@@@@CCAACCCCCC	XA:Z:2,-65308958,44M2D23M,2;	MC:Z:70M	MD:Z:67	PG:Z:MarkDuplicates	RG:Z:tumor_62JREAAXX_8	NM:i:0	AS:i:67	XS:i:59


Check initial variant call from Mutect2

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

less pairedVariants_mutect2.vcf | egrep '##' | less -S

pairedVariants_mutec2 contains no filter information. Filter information comes from each variant and generates tags.

## Filtering

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

gatk FilterMutectCalls \
-R references/human_g1k_v37.fasta \
--filtering-stats pairedVariants_mutect2.vcf.stats \
-V pairedVariants_mutect2.vcf \
-O pairedVariants_mutect2_filtered.vcf

Check filtered results

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

less -S pairedVariants_mutect2_filtered.vcf | egrep -v '#' | less -S

Since we are comparing different methods, we want to make sure to consider all sites. Hence we will split multiallelic variants into separate variants. Moreover, we will make sure everything is left-aligned.

In [ ]:
# splitting multiallelic variants
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

bcftools norm \
-m-both \
-f references/human_g1k_v37.fasta \
-Oz \
-o pairedVariants_mutect2_filtered_normalized.vcf.gz pairedVariants_mutect2_filtered.vcf


Lines   total/split/realigned/skipped:	114/4/5/0


In [ ]:
# aligning left
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

tabix pairedVariants_mutect2_filtered_normalized.vcf.gz 

[tabix] the index file exists. Please use '-f' to overwrite.


# Running varscan2
Another method to call somatic variants is varscan2.

Create pileup for bam files

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

samtools mpileup \
-B \
-q 1 \
-f references/human_g1k_v37.fasta \
-r 9:130215000-130636000 \
alignments/tumor/tumor.sorted.realigned.bam > tumor.mpileup

samtools mpileup \
-B \
-q 1 \
-f references/human_g1k_v37.fasta \
-r 9:130215000-130636000 \
alignments/normal/normal.sorted.realigned.bam > normal.mpileup

[W::hts_idx_load2] The index file is older than the data file: alignments/tumor/tumor.sorted.realigned.bai
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
[W::hts_idx_load2] The index file is older than the data file: alignments/normal/normal.sorted.realigned.bai
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


Check pileup file

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

less -S normal.mpileup

Run varscan2

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

varscan \
somatic normal.mpileup tumor.mpileup \
--output-vcf 1 \
--strand-filter 1 \
--somatic-p-value 0.05 \
--output-snp varscan2.snp.vcf \
--output-indel varscan2.indel.vcf

Check results

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

less -S varscan2.snp.vcf 

Set minimum variant allele frequency to 0.05%. Resulting 'hc' files are 'high-confidence'.

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

varscan processSomatic varscan2.indel.vcf --min-tumor-freq 0.05
varscan processSomatic varscan2.snp.vcf --min-tumor-freq 0.05

79 VarScan calls processed
3 were Somatic (1 high confidence)
75 were Germline (73 high confidence)
1 were LOH (1 high confidence)
447 VarScan calls processed
14 were Somatic (13 high confidence)
426 were Germline (411 high confidence)
7 were LOH (7 high confidence)


Reading input from varscan2.indel.vcf
Opening output files: varscan2.indel.Somatic.vcf varscan2.indel.Germline.vcf varscan2.indel.LOH.vcf 
Reading input from varscan2.snp.vcf
Opening output files: varscan2.snp.Somatic.vcf varscan2.snp.Germline.vcf varscan2.snp.LOH.vcf 


Zip, index, and combine high confidence variants from snps and indels

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

bgzip varscan2.snp.Somatic.hc.vcf
tabix varscan2.snp.Somatic.hc.vcf.gz

bgzip varscan2.indel.Somatic.hc.vcf
tabix varscan2.indel.Somatic.hc.vcf.gz

bcftools concat \
-Oz \
-a \
-r 9:130215000-130636000 \
varscan2.snp.Somatic.hc.vcf.gz varscan2.indel.Somatic.hc.vcf.gz \
-o pairedVariants_varscan2_filtered.vcf.gz

Normalize (as done for mutatec2)

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

bcftools norm \
-m-both \
-f references/human_g1k_v37.fasta \
-Oz \
-o pairedVariants_varscan2_filtered_normalized.vcf.gz pairedVariants_varscan2_filtered.vcf.gz

tabix pairedVariants_varscan2_filtered_normalized.vcf.gz

Lines   total/split/realigned/skipped:	14/0/0/0


# Compare variant callers

Combine output of both variant callers

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

bcftools isec \
-n+1 \
-f PASS \
-p intersectionDirectory pairedVariants_mutect2_filtered_normalized.vcf.gz pairedVariants_varscan2_filtered_normalized.vcf.gz

Check Results

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/intersectionDirectory/

less -S sites.txt

9	130223126	C	T	01
9	130244419	CTTT	C	10
9	130264057	T	A	11
9	130296899	A	T	11
9	130308743	C	A	11
9	130308789	C	T	11
9	130310160	C	T	11
9	130310330	C	G	11
9	130316821	C	T	01
9	130337805	CT	C	01
9	130505445	G	GGCAGGGGCTGCGTCTTTCTAGTGAGTTCC	10
9	130602324	C	A	11
9	130633690	C	T	11
9	130634110	C	T	11
9	130634993	C	T	10
9	130635011	C	G	10
9	130635494	C	T	11
9	130635520	C	G	11
9	130635680	T	TGTAGAGGGAAGAAGGAA	10
9	130635681	C	CCAGGAAGATGGCTGGATAGT	10


Check summary results

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/intersectionDirectory/

less -S sites.txt | cut -f  5 | sort | uniq -c 

      3 01
      6 10
     11 11


These results indicate, that varscan has 3 unique variants, muteect2 has 6 unique variants and 11 variants are shared.

Merge both vcf files

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

bcftools merge \
-f PASS \
pairedVariants_mutect2_filtered_normalized.vcf.gz pairedVariants_varscan2_filtered_normalized.vcf.gz \
-o pairedVariants_mutect2_varscan2.vcf.gz \
-Oz

[W::bcf_hdr_merge] Trying to combine "AD" tag definitions of different lengths


Check combined file

In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

zless -S pairedVariants_mutect2_varscan2.vcf.gz | egrep '##' | head -n 10

zless -S pairedVariants_mutect2_varscan2.vcf.gz | egrep -v '#' | head -n 1

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FILTER=<ID=FAIL,Description="Fail the site if all alleles fail but for different reasons.">
##FILTER=<ID=base_qual,Description="alt median base quality">
##FILTER=<ID=clustered_events,Description="Clustered events observed in the tumor">
##FILTER=<ID=contamination,Description="contamination">
##FILTER=<ID=duplicate,Description="evidence for alt allele is overrepresented by apparent duplicates">
##FILTER=<ID=fragment,Description="abs(ref - alt) median fragment length">
##FILTER=<ID=germline,Description="Evidence indicates this site is germline, not somatic">
##FILTER=<ID=haplotype,Description="Variant near filtered variant on same haplotype.">
9	130223126	.	C	T	.	PASS	SOMATIC;SS=2;SSC=18;GPV=1;SPV=0.014587;DP=55	GT:GQ:DP:RD:AD:FREQ:DP4	./.:.:.:.:.:.:.	./.:.:.:.:.:.:.	0/0:.:26:26:0:0%:14,12,0,0	0/1:.:29:22:6:21.43%:15,7,1,5


# ANNOTATION NOT FINISHED (NO ACCESS TO ANNOVAR)

Annotating merged variants file

In [ ]:
# download annovar
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

mkdir annovar

wget http://www.openbioinformatics.org/annovar/download/annovar.latest.tar.gz.mirror
tar xzvf annovar.tar.gz

--2022-01-03 20:20:41--  http://www.openbioinformatics.org/annovar/download/annovar.latest.tar.gz.mirror
Resolving www.openbioinformatics.org (www.openbioinformatics.org)... 67.205.156.247
Connecting to www.openbioinformatics.org (www.openbioinformatics.org)|67.205.156.247|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-01-03 20:20:41 ERROR 404: Not Found.

tar (child): annovar.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants

mkdir results

/usr/local/annovar/table_annovar.pl \
pairedVariants_mutect2_varscan2.vcf.gz /usr/local/annovar/humandb/ \
-buildver hg19 \
-out results/annotated_mutect2_varscan2 \
-remove \
-protocol refGene \
-operation g \
-nastring . \
--vcfinput

mkdir: cannot create directory ‘results’: File exists


# Copy number variations

In this workshop, we will present the main steps for calling copy number variations (CNVs). Normally we would perform a copy number variation analysis on alignment files (BAMs). Due to time and resource constraints we will not be able to do this analysis on the full sequence or from shortened bam files. Instead we will be working with some preprocessed data using gc correction files and mpileups. Our data sample is again the cagekid sample c0098.

In [ ]:
# generate configuration file for free-c
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants
mkdir copynumber
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/copynumber

bash /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/scripts/generate_controlfreec_configurationfile.sh > CBW_config_1.txt

less CBW_config.txt

# this file is good for the course, but will not work in my case. I will modify the CBW_config file outside this notebook.

In [ ]:
# run control-freec
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/copynumber

freec -conf CBW_config_1.txt > output.log

[E::hts_hopen] fail to open file '/content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/alignments/controlfreec_alignments/CBW_regions_c0098_Tumor.sorted.markduplicates.bam'
[E::hts_open_format] fail to open file '/content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/alignments/controlfreec_alignments/CBW_regions_c0098_Tumor.sorted.markduplicates.bam'
[mpileup] failed to open /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/alignments/controlfreec_alignments/CBW_regions_c0098_Tumor.sorted.markduplicates.bam: Invalid argument
[mpileup] 1 samples in 1 input files
[E::hts_hopen] fail to open file '/content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/alignments/controlfreec_alignments/CBW_regions_c0098_Tumor.sorted.markduplicates.bam'
[E::hts_open_format] fail to open file '/content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/alignments/controlfreec_

In [ ]:
# annotating copy number calls
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/accessory_files/controlfreec_files

bedtools intersect -wb -b <(less CBW_regions_c0098_Tumor.sorted.markduplicates.bam_sample.cpn_CNVs | awk 'NF==7' | awk '{print "chr"$1"\t"$2"\t"$3"\t"$0}' | less -S) -a <(less /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/accessory_files/Homo_sapiens.GRCh37.Ensemble100.FullGeneAnnotations.txt | awk '$4=="ensembl_havana"') | awk '{print $1"\t"$2"\t"$3"\t"$7"\t"$8"\t"$15}' > AnnotatedCBW_regions_c0098_Tumor.sorted.markduplicates.bam_CNVs.tsv

less -S AnnotatedCBW_regions_c0098_Tumor.sorted.markduplicates.bam_CNVs.tsv 

In [ ]:
# examine a gene of interest, such as a loss of VHL which is common in kidney cancers
%%bash
cd /content/drive/MyDrive/Work/13_CancerAnalysisCourse/labs/Module4_SomaticVariants/accessory_files/controlfreec_files

less AnnotatedCBW_regions_c0098_Tumor.sorted.markduplicates.bam_CNVs.tsv | awk '$4=="VHL"' | less -S


chr3	10141778	10153667	VHL	protein_coding	2


## Data Visualization

For results of data visualization please refer to separate R notebook.